In [1]:
import gensim.downloader as api
import gensim
import pandas as pd
from nltk.corpus import stopwords
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from gensim import utils
from scipy.spatial.distance import cdist
import os

stop_words = stopwords.words('english')



In [104]:
print(f"___________________________ {os.getcwd()} ________________________________")

NameError: name 'os' is not defined

In [80]:
path = api.load("word2vec-google-news-300", return_path=True)
print(path)

C:\Users\Giannis/gensim-data\word2vec-google-news-300\word2vec-google-news-300.gz


In [90]:
g_model = gensim.models.KeyedVectors.load_word2vec_format("C:/Users/Giannis/gensim-data/word2vec-google-news-300/GoogleNews-vectors-negative300.bin", binary=True)  

In [2]:
glove_model = api.load("glove-wiki-gigaword-50")

In [95]:

# preprocess the lyrics column of the dataframe by converting to lowercase and removing punctuation
def preprocess_lyrics(df):
    df["lyrics"] = [" ".join(utils.simple_preprocess(x)) for x in df["lyrics"]]
    return df

def encode_lyrics(df):
    embeddings = []
    for lyrics in df["lyrics"]:
        embedding = []
        for word in lyrics.split():
            try:
                embedding.append(g_model[word])
            except KeyError:
                pass
        embeddings.append(np.mean(embedding, axis=0))
    return np.array(embeddings)

def save_encoded_df(df, filename):
    np.save(filename, encode_lyrics(preprocess_lyrics(df)))

def load_encoded_df(filename):
    return np.load(filename)

def find_similar_songs(query, encoded_df, df, batch_size=1000):
    df_query = pd.DataFrame({"lyrics": [query]})
    # processed_df_query = preprocess_lyrics(df_query)
    query_embedding = encode_lyrics(df_query)
    distances= cdist(encoded_df, query_embedding)
    smallest_distances = np.argsort(distances.T)[0][:10]
    results = [] 
    # print(smallest_distances[0].shape)
    for distance in smallest_distances:
        results.append(df.iloc[distance])
    return results



In [102]:
df = pd.read_csv("C:/Users/Giannis/Desktop/SearchEngine/data/clean_songs.tsv", sep="\t")

# save the encoded dataframe to a file
save_encoded_df(df, "encoded_data.npy")

In [103]:

encoded_df = load_encoded_df("encoded_data.npy")

query = "imagine all the people"
results = find_similar_songs(query, encoded_df, df)

results_dict = {"artist":[], "title":[], "lyrics":[]}
for result in results:
    results_dict["artist"].append(result["artist"])
    results_dict["title"].append(result["title"])
    results_dict["lyrics"].append(result["lyrics"])
    # df_results.append({"artist":result["artist"], "title":result["title"], "lyrics":result["lyrics"]}, ignore_index=True)
    # print(f'Artist:{result["artist"]}\tTitle: {result["title"]}\tLyrics: {result["lyrics"]}')

df_results = pd.DataFrame(results_dict)
df_results.head()


,artist,title,lyrics
0,R. Kelly,Imagine That,close your eyes now relax your mind now want y...
1,Bon Jovi,Imagine,imagine there no heaven it easy if you try no ...
2,Our Lady Peace,Imagine,imagine there no heaven it easy if you try no ...
3,Diana Ross,Imagine,imagine there no heaven it easy if you try no ...
4,Avril Lavigne,Imagine,imagine there no heaven it easy if you try no ...


In [3]:
 load the encoded dataframe from a file
encoded_df = load_encoded_df("encoded_data.npy")

# find the best 10 results based on cosine distance
query = "imagine all the people"
results = find_similar_songs(query, encoded_df, df)